In [11]:
# load api key from .env
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.environ.get('API_KEY')

In [13]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    document = fitz.open(pdf_path)

    # Iterate through each page
    for page_num in range(len(document)):
        if page_num == 1:
            # Get the page
            page = document.load_page(page_num)
            building_to_street = {}

            # Extract text from the page
            text = page.get_text()
            lines = text.split('\n')

            # Print the text of the page
            # print(f"Page {page_num + 1}:\n{text}\n")

            for k, v in enumerate(lines):
                # if line starts with number, add previous line to dictionary
                # as the building name and the current line as the street address
                if len(v) == 0:
                    continue
                if v[0].isdigit():
                    building_name = lines[k-1]
                    street_address = v
                    building_to_street[building_name] = street_address

    # Close the document
    document.close()
    return building_to_street

building_to_street = extract_text_from_pdf('umkc-volker-campus-map-0823.pdf')

In [15]:
import requests
import json

def geocode_address(api_key, address):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params={'address': address, 'key': api_key})
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
    return None, None

# Example usage
for building_name, street_address in building_to_street.items():
    latitude, longitude = geocode_address(api_key, street_address + ', Kansas City, MO')
    building_to_street[building_name] = f"{latitude}, {longitude}"
    # save building_name: latitude, longitude to a json file
with open('building_to_latlong.json', 'w') as f:
    json.dump(building_to_street, f, indent=4)